
##### Version for single obj using the three separate networks for prediction.



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset as dset
from torch.utils.data import DataLoader
from scipy.io import loadmat
from PIL import Image
import matplotlib.pyplot as plt
from logger import Logger
import os as os
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
torch.backends.cudnn.deterministic=True

plt.rcParams.update({'font.size': 15,'lines.markersize':10})
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']="0"

In [ ]:
from utils_single_point import *
from visualize import *
np.random.seed(190)
torch.manual_seed(120) ##set the random seed so when construct the 
#os.chdir('E:/Academic/MATLAB/Dehui_FS/2018_10_24 data')
#os.chdir('E:/Academic/MATLAB/Dehui_FS/2018_11_03 data')
#os.chdir('E:/Academic/MATLAB/Dehui_FS/2018_11_21 CCD XYZ sweep')
os.chdir('/home/zyhuang/WD/Net3D/2018_11_21 CCD XYZ sweep/')
os.getcwd()

In [ ]:
logfreq=10
lr_x,lr_y,lr_z=1e-2,1e-2,1e-2
bs_train=50
bs_test=200
num_epoch=2000
dataset_name = 'FS_32by32_CMOS_avg_win1_normalizedbymax'
save_name=dataset_name+'/Depthnet4_4_3(128_128_32features)_adam_lr_x_'+str(lr_x)+'_lr_y_'+str(lr_y)+'_lr_z_'+str(lr_z)+'_bs_train_'+str(bs_train)+'_bs_test_'+str(bs_test)+'_numepoch_'+str(num_epoch)
logger = Logger('./logs_paper_data/'+ save_name) ##set the logger for tensorboard and save the log file to ./logs/Adam 

"""
dataidx:specify what subset of the fulldata is used to split into train/test, 
e.g, for a particular fixed xy and sweep z,np.arange(0,1999,100)
for a particular fixed z and sweep xy, np.arange(100,200,1)
test+idx specify among the subset of the fulldata, which point is used as the testset (start from 1, ie, test_idx=1 means FS_all[0] )
"""
#data_idx=np.arange(0,396,36)## fixed  particular xy, sweep z 
#dataidx=np.arange(0,6) ## fixed  particular y，z, sweep x
data_idx=np.arange(0,1331) ##all data used for 3D regression
test_idx=(np.random.permutation(np.arange(1331))+1)[:bs_test] ##+1 since test_idx start from 1

In [ ]:

Full_dataset=FSdataset(dataset_name+'.mat',set='Full',data_idx=data_idx,test_idx=test_idx)

train_dataset=FSdataset(dataset_name+'.mat',set='Train',data_idx=data_idx,test_idx=test_idx)
train_loader=DataLoader(train_dataset, batch_size=bs_train,shuffle=True)
test_dataset=FSdataset(dataset_name+'.mat',set='Test',data_idx=data_idx,test_idx=test_idx)
test_loader=DataLoader(test_dataset, batch_size=bs_test,shuffle=False)

In [ ]:
#dtype = torch.FloatTensor ##run on cpu
dtype = torch.cuda.FloatTensor 

net_x=depthnet4(input_dim=32*32*2,num_features=128)
net_y=depthnet4(input_dim=32*32*2,num_features=128)
net_z=depthnet3(input_dim=32*32*2,num_features=32)

net_x.type(dtype)
net_y.type(dtype)
net_z.type(dtype)

#criterion=torch.nn.L1Loss()
criterion=torch.nn.MSELoss() #Assignment2 moves this to gpu as well, is it necessary? For MSE loss, no since it has no parameter
optimizer_x = torch.optim.Adam(net_x.parameters(), lr=lr_x)
optimizer_y = torch.optim.Adam(net_y.parameters(), lr=lr_y)
optimizer_z = torch.optim.Adam(net_z.parameters(), lr=lr_z)
#optimizer = torch.optim.SGD(net.parameters(), lr=1e-5)

In [ ]:
loss_x_history=[]
loss_y_history=[]
loss_z_history=[]
test_loss_x_history=[]
test_loss_y_history=[]
test_loss_z_history=[]

for epoch in range(num_epoch):
    if np.mod(epoch,logfreq)==0: 
        print("Current epoch number%d" %epoch) 

    for idx,data in enumerate(train_loader,0):
        net_x.train()
        net_y.train()
        net_z.train()
        FS_bat=data['FS']
        trueXobj_bat,trueYobj_bat,trueZobj_bat=data['xyz'][:,0],data['xyz'][:,1],data['xyz'][:,2]
        FS_bat_var,trueXobj_bat_var,trueYobj_bat_var,trueZobj_bat_var=Variable(FS_bat.type(dtype)),Variable(trueXobj_bat.type(dtype)),Variable(trueYobj_bat.type(dtype)),Variable(trueZobj_bat.type(dtype))
        estXobj,estYobj,estZobj=net_x(FS_bat_var),net_y(FS_bat_var),net_z(FS_bat_var)
        loss_x,loss_y,loss_z=criterion(estXobj,trueXobj_bat_var),criterion(estYobj,trueYobj_bat_var) ,criterion(estZobj,trueZobj_bat_var)  #the criterion calculate total MSE/Batch size 
        loss_x_history.append(loss_x.data[0])
        loss_y_history.append(loss_y.data[0])
        loss_z_history.append(loss_z.data[0])
        if np.mod(epoch,logfreq)==0: 
            print('loss_x is %3f,loss_y is %3f,loss_z is %3f'%(loss_x,loss_y,loss_z))


        optimizer_x.zero_grad()
        optimizer_y.zero_grad()
        optimizer_z.zero_grad()
        loss_x.backward()
        loss_y.backward()
        loss_z.backward()
        optimizer_x.step()
        optimizer_y.step()
        optimizer_z.step()
          
    
    if np.mod(epoch,logfreq)==0:  
        net_x.eval()
        net_y.eval()
        net_z.eval()
        data=next(iter(test_loader))
        FS_test_bat,trueXobj_test_bat,trueYobj_test_bat,trueZobj_test_bat=data['FS'],data['xyz'][:,0],data['xyz'][:,1],data['xyz'][:,2]
        estXobj_test,estYobj_test,estZobj_test=net_x(Variable(FS_test_bat.type(dtype))),net_y(Variable(FS_test_bat.type(dtype))),net_z(Variable(FS_test_bat.type(dtype)))
        test_loss_x,test_loss_y,test_loss_z=criterion(estXobj_test,Variable(trueXobj_test_bat.type(dtype))),criterion(estYobj_test,Variable(trueYobj_test_bat.type(dtype))),criterion(estZobj_test,Variable(trueZobj_test_bat.type(dtype)))
        
        test_loss_x_history.append(test_loss_x.data[0])
        test_loss_y_history.append(test_loss_y.data[0])
        test_loss_z_history.append(test_loss_z.data[0])
        
        #print('Test Loss is %3f test prediction is %3f.' %(test_loss.data[0],estZobj_test.data[0]))#for one test sample
        print('Test Loss_x is %3f.,Test Loss_y is %3f.,Test Loss_z is %3f.' %(test_loss_x.data[0],test_loss_y.data[0],test_loss_z.data[0]))#for multiple test sample
        
        step=epoch+1
        info = {
        'training_loss_x': loss_x.data[0],
        'training_loss_y': loss_y.data[0],
        'training_loss_z': loss_z.data[0],
        'val_loss_x':test_loss_x.data[0],
        'val_loss_y':test_loss_y.data[0],
        'val_loss_z':test_loss_z.data[0],
            
        #'Zobj_est':estZobj_test.data[0]##for one test sample
            
        }
   
        for tag, value in info.items():
            logger.scalar_summary(tag, value, step+1)        

In [ ]:
torch.save(net_x.state_dict(), os.path.join('logs_paper_data',save_name, 'model_x.pth'))
torch.save(net_y.state_dict(), os.path.join('logs_paper_data',save_name, 'model_y.pth'))
torch.save(net_z.state_dict(), os.path.join('logs_paper_data',save_name, 'model_z.pth'))

In [ ]:
plt.figure()
plt.semilogy(loss_x_history[::len(train_loader)],label='training_loss_x')#training_loss
plt.semilogy(loss_y_history[::len(train_loader)],label='training_loss_y')#training_loss
plt.semilogy(loss_z_history[::len(train_loader)],label='training_loss_z')#training_loss
plt.legend()
plt.title('Training loss vs epoch number')
plt.savefig('logs_paper_data/'+save_name+'/Training loss')

plt.figure()
plt.semilogy(np.arange(0,epoch,logfreq),test_loss_x_history[:],label='test_loss_x')
plt.semilogy(np.arange(0,epoch,logfreq),test_loss_y_history[:],label='test_loss_y')
plt.semilogy(np.arange(0,epoch,logfreq),test_loss_z_history[:],label='test_loss_z')
plt.legend()
plt.title('Test loss vs epoch number')
plt.savefig('logs_paper_data/'+save_name+'/Test loss')

## Test

In [ ]:

#this part of code is added , previsouly the test loss is not the final model test loss since np.mod(epoch,logfreq)==0 is false for last epoch 
net_x.eval()
net_y.eval()
net_z.eval()
data=next(iter(test_loader))
FS_test_bat,trueXobj_test_bat,trueYobj_test_bat,trueZobj_test_bat=data['FS'],data['xyz'][:,0],data['xyz'][:,1],data['xyz'][:,2]
estXobj_test,estYobj_test,estZobj_test=net_x(Variable(FS_test_bat.type(dtype))),net_y(Variable(FS_test_bat.type(dtype))),net_z(Variable(FS_test_bat.type(dtype)))
test_loss_x,test_loss_y,test_loss_z=criterion(estXobj_test,Variable(trueXobj_test_bat.type(dtype))),criterion(estYobj_test,Variable(trueYobj_test_bat.type(dtype))),criterion(estZobj_test,Variable(trueZobj_test_bat.type(dtype)))
##

test_bat=next(iter(test_loader))
FS_bat,trueXYZobj_bat=test_bat['FS'],test_bat['xyz'][:,:]
predXYZobj_bat=torch.cat((net_x(Variable(FS_bat.type(dtype))),net_y(Variable(FS_bat.type(dtype))),net_z(Variable(FS_bat.type(dtype)))),dim=1).data.cpu()
print(predXYZobj_bat)
print(trueXYZobj_bat)

show3Dpred(trueXYZobj_bat,predXYZobj_bat,howmany=10,save_name='logs_paper_data/'+save_name+'/3D')
plt.savefig('3d')
show2D(trueXYZobj_bat,predXYZobj_bat,howmany=10,save_name='logs_paper_data/'+save_name+'/2Ds')
plt.savefig('2d')
text_file = open('logs_paper_data/'+save_name+"/Loss_final.txt", "w")
text_file.write('loss_x is %.4f(%.3f mm),loss_y is %.4f(%.3f mm),loss_z is %.4f(%.3f mm)\n'%(loss_x.data[0],np.sqrt(loss_x.data[0]),loss_y,np.sqrt(loss_y.data[0]),loss_z,np.sqrt(loss_z.data[0])))
text_file.write('Test Loss_x is %.4f(%.3f mm).,Test Loss_y is %.4f(%.3f mm).,Test Loss_z is %.4f(%.3f mm).' %(test_loss_x.data[0],np.sqrt(test_loss_x.data[0]),test_loss_y.data[0],np.sqrt(test_loss_y.data[0]),test_loss_z.data[0],np.sqrt(test_loss_z.data[0])))
text_file.close()

In [ ]:
# Time benchmark part
# res=4
# net_x=depthnet4(input_dim=res*res*2,num_features=128)
# net_y=depthnet4(input_dim=res*res*2,num_features=128)
# net_z=depthnet3(input_dim=res*res*2,num_features=32)

# net_x.type(dtype)
# net_y.type(dtype)
# net_z.type(dtype)
# #this part of code is added , previsouly the test loss is not the final model test loss since np.mod(epoch,logfreq)==0 is false for last epoch 
# net_x.eval()
# net_y.eval()
# net_z.eval()

# test_bat=next(iter(test_loader))
# FS_bat=torch.zeros(1,2,res,res)
# times = []
# for _ in range(500):
#     start = torch.cuda.Event(enable_timing=True)
#     end = torch.cuda.Event(enable_timing=True)
#     start.record()
#     with torch.no_grad():
#         predXYZobj_bat=torch.cat((net_x(Variable(FS_bat.type(dtype))),net_y(Variable(FS_bat.type(dtype))),net_z(Variable(FS_bat.type(dtype)))),dim=1)
#     end.record()
#     torch.cuda.synchronize()
#     #print(start.elapsed_time(end))
#     times.append(start.elapsed_time(end))
# print('Mean is {0:.3f}'.format(np.mean(times)))

In [ ]:
text_file = open('logs_paper_data/'+save_name+"/Loss_final.txt", "w")
text_file.write('loss_x is %.4f(%.3f mm),loss_y is %.4f(%.3f mm),loss_z is %.4f(%.3f mm)\n'%(loss_x.data[0],np.sqrt(loss_x.data[0]),loss_y,np.sqrt(loss_y.data[0]),loss_z,np.sqrt(loss_z.data[0])))
text_file.write('Test Loss_x is %.4f(%.3f mm).,Test Loss_y is %.4f(%.3f mm).,Test Loss_z is %.4f(%.3f mm).' %(test_loss_x.data[0],np.sqrt(test_loss_x.data[0]),test_loss_y.data[0],np.sqrt(test_loss_y.data[0]),test_loss_z.data[0],np.sqrt(test_loss_z.data[0])))
text_file.close()

In [ ]:
#Saving test data as .mat
from scipy import io
data={'trueXYZ':trueXYZobj_bat.cpu().numpy(), 'PredictedXYZ':predXYZobj_bat.cpu().numpy()}
io.savemat('logs_paper_data/'+save_name+"/testData.mat",data)

## Some plots

In [ ]:

print('Training FS')
showFS(train_dataset)
print('Test FS')
showFS(test_dataset)

#plot of normalized current vs z for different pixels
for i in range(4):
    for j in range(4):
        plt.plot(train_dataset[:]['FS'][:,1,i,j]/train_dataset[:]['FS'][:,1,i,j].sum())
plt.title('Plot of normalized current vs Z for each pixel in back plane')
plt.show()
#plot of normalized current vs z for different pixels
for i in range(4):
    for j in range(4):
        plt.plot(train_dataset[:]['FS'][:,0,i,j]/train_dataset[:]['FS'][:,0,i,j].sum())
plt.title('Plot of normalized current vs Z for each pixel in front plane')
plt.show()

In [ ]:
plt.imshow(net.FC_front.weight.view(4,4).data.abs().cpu())
plt.show()
plt.plot(train_dataset[:]['FS'][:,0,0,1]/train_dataset[:]['FS'][:,0,0,1].sum())

In [ ]:
plt.imshow(net.FC_back.weight.view(4,4).data.abs().cpu())
plt.show()
plt.plot(train_dataset[:]['FS'][:,1,2,2]/train_dataset[:]['FS'][:,1,2,2].sum())

In [ ]:
X_sweep_trace(Full_dataset,Z_idx=10,Y_idx=6,nX=11,nY=11,nZ=11,H=4,W=4)
Y_sweep_trace(Full_dataset,Z_idx=10,X_idx=6,nX=11,nY=11,nZ=11,H=4,W=4)

In [ ]:
#to test if any test sample is also from train data
((test_dataset[:]['xyz'][6]==train_dataset[:]['xyz']).sum(axis=1)==3).sum()